# Model Monitoring (beta)

```{note}
The model monitoring feature is based on Iguazio's streaming technology. To enable Model Monitoring you will need to contact Iguazio.
```

## Introduction
MLRun provides a model monitoring service that tracks the performance of models in production to help identify
potential issues with concept drift and prediction accuracy before they impact business goals.
Typically, Model monitoring is used by devops for tracking model performance, and by data scientists to track model drift.
Two monitoring types are supported:
1. Model operational performance (stream processing)
2. Concept drift detection and accuracy monitoring (features/labels/predictions)

Model Monitoring also provides warning alerts that can be sent to stakeholders for processing.

The Model Monitoring feature data can be viewed using Iguazio's user interface portal or through Grafana Dashboards.

## Architecture
The Model Monitoring process flow starts by collecting operational data, converts them to vectors which are then posted to the Model Server.
Then, the Model Server is wrapped around a machine learning model that uses a function to calculate predictions based on the available vectors.
Next, the Model Server creates a log for the input and output of the vectors, and the entries are written to the production data stream.
While the Model Server is processing the vectors, a Nuclio operation monitors the log of the data stream and is triggered when a new log entry is detected.
The Nuclio function examines the log entry, processes it in to statistics which are then written to the statistics databases.
In parallel, a scheduled MLRun runs and reads the parquet files performing the drift analysis which is then stored so that
the user can retrieve the analysis data in the Iguazio UI or in a Grafana instance.

![Architecture](./model-monitoring-data-flow.svg)

## Model Endpoints Overview Dashboard
The Overview dashboard will display the model endpoint IDs of a specific project. Only deployed models with Model Monitoring option enabled are displayed.
Endpoint IDs are URIs used to provide access to performance data, drift detection statistics, and accuracy monitoring of a deployed model.

![overview](./overview.png)

The Overview screen providers details about the performance of all the deployed and monitored models within a project. You can change projects by choosing a new project from the
**Project** dropdown. The Overview dashboard displays the number of endpoints in the project, the average predictions per second (using a 5-minute rolling average),
the average latency (using a 1-hour rolling average), and the total error count in the project.

Additional details include:
* **Endpoint ID**&mdash;the ID of the deployed model. Use this link to drill down to the model performance and details screens.
* **Function**&mdash;the MLRun function to access the model
* **Model**&mdash;user defined name for the model
* **Model Class**&mdash;the implementation class that is used by the endpoint
* **First Request**&mdash;first request for production data
* **Last Request**&mdash;most recent request for production data
* **Error count**&mdash;total error count from processing production data
* **Accuracy**&mdash;a numeric value representing the accuracy of model predictions
* **Drift Status**&mdash;no drift (green), possible drift (yellow), drift detected (red)

At the bottom of the dashboard are heat maps for the Predictions per second, Average Latency and Errors. The heat maps display data based on 15 minute intervals.

Click an endpoint ID to drill down the performance details of that model.

## Model Endpoint Details Dashboard
The model endpoint details dashboard displays the real time performance data of the selected model in detail.
Model performance data provided is rich and is used to fine tune or diagnose potential performance issues that may affect business goals.
The data in this dashboard changes based on the selection of the project and model.

This dashboard is broken down into three panes:

1. [Project and model summary](#project-and-model-summary)
2. [Analysis panes](#analysis-panes)
   1. Overall drift analysis
   2. Features analysis
3. [Incoming features graph](#incoming-features-graph)

![details](./details.png)

### Project and Model Summary
Use the dropdown to change the project and model. The dashboard presents the following information about the project:
* **Endpoint ID**&mdash;the ID of the deployed model
* **Model**&mdash;user defined name for the model
* **Function URI**&mdash;the MLRun function to access the model
* **Model Class**&mdash;the implementation class that is used by the endpoint
* **Prediction/s**&mdash;the average number of predictions per second over a rolling 5-minute period
* **Average Latency**&mdash;the average latency over a rolling 1-hour period
* **First Request**&mdash;first request for production data
* **Last Request**&mdash;most recent request for production data


Use the [Performance](#model-endpoint-performance-dashboard) and [Overview](#model-endpoints-overview-dashboard) buttons view those dashboards.

### Analysis Panes
This pane is broken down into sections: Overall Drift Analysis and Features Analysis.
Concept Drift in machine learning is a situation where the statistical properties of the target variable (what the model is trying to predict) change over time.
In other words, the production data has changed significantly over the course of time and no longer matches the input data used to train the model.
So, for this new data, accuracy of the model predictions is low.
For more information see <a href="https://www.iguazio.com/glossary/concept-drift/" target="_blank">Concept Drift</a>.

The Overall Drift Analysis pane provides performance statistics for the currently selected model. This pane includes three types of statistics:
* **Total Variation Distance** (TVD sum and mean)&mdash;this is the statistical difference between the actual predictions, and the model's trained predictions
* **Hellinger Distance** (sum and mean)&mdash;this is a type of f-divergence that quantifies the similarity between the actual predictions, and the model's trained predictions.
* **Kullback–Leibler Divergence** (KLD sum and mean)&mdash;this is the measure of how the probability distribution of actual predictions is different from the second, model's trained reference probability distribution.

The Features Analysis pane provides details of the drift analysis for each feature in the selected model.
This pane includes five types of statistics:
* **Actual** (min, mean and max)&mdash;results based on actual live data stream
* **Expected** (min, mean and max)&mdash;results based on training data
* **Total Variation Distance** (TVD)&mdash;this is the statistical difference between the actual predictions, and the model's trained predictions
* **Hellinger Distance**&mdash;this is a type of f-divergence that quantifies the similarity between the actual predictions, and the model's trained predictions.
* **Kullback–Leibler Divergence** (KLD)&mdash;this is the measure of how the probability distribution of actual preditions is different from the second, model's trained reference probability distribution.

### Incoming Features Graph
This graph displays the performance of the features that are in the selected model based on sampled data points from actual feature production data.
The graph displays the values of the features in the model over time.

## Model Endpoint Performance Dashboard
Model endpoint performance displays performance details in graphical format.

![performance](./performance.png)

This dashboard is broken down into 5 graphs:
* **Drift Measures**&mdash;the overall drift over time for each of the endpoints in the selected model
* **Average Latency**&mdash;the average latency of the model in 5 minute intervals, for 5 minutes and 1 hour rolling windows
* **Predictions/s**&mdash;the model predictions per second displayed in 5 second intervals for 5 minutes (rolling)
* **Predictions Count**&mdash;the number of predictions the model makes for 5 minutes and 1 hour rolling windows
* **Custom Metrics**&mdash;

### Configuring Custom Metrics <a name="config-custom-metrics"></a>
The Custom Metrics graph can be used to view metrics that are not typically displayed on the current suite of dashboards,
Additional metrics include:
* TVD&mdash;

## Initial Set Up (pre-requisites)
1. Make sure you have the `mlrun-api` as a Grafana data source configured in your Grafana instance. If not,
add it by:
   1. Open your grafana instance.
   2. Navigate to `Configuration -> Data Sources`.
   3. Press `Add data source`.
   4. Select the `SimpleJson` datasource and configure the following parameters.
    ```
    Name: mlrun-api
    URL: http://mlrun-api:8080/api/grafana-proxy/model-endpoints
    Access: Server (default)

    ## Add a custom header of:
    X-V3io-Session-Key: <YOUR ACCESS KEY>
    ```
    5. Press `Save & Test` for verification. A confirmation message should appear if it is working.

2. Download the following monitoring dashboards:
    1. [Overview](./dashboards/overview.json)
    2. [Details](./dashboards/details.json)
    3. [Performance](./dashboards/performance.json)

3. Import the downloaded dashboards to your Grafana instance.

Use the following code to supply training data to the model in order to utilize drift measurement.

4. To allow the system to utilize drift measurement, make sure you supply the train set when logging the model in the
   training step

    ```python
    # Log model
    context.log_model(
        "model",
        body=dumps(model),
        artifact_path=context.artifact_subpath("models"),
        extra_data=eval_metrics,
        model_file="model.pkl",
        metrics=context.results,
        training_set=<TRAIN_SET>,  # <-
        label_cols=<LABEL_COLS>,  # <-
        labels={"class": "sklearn.linear_model.LogisticRegression"}
    )
    ```
5. When serving the model, make sure that the Nuclio function is deployed with tracking enabled by applying
   `fn.set_tracking()` on the serving function.

## Configuration
The stream processing portion of the model monitoring can be deployed under multiple configuration options. The
available configurations can be found under `stream.Config`. Once configured, it should be supplied as environment
parameters to the Nuclio function by setting `fn.set_envs`

```python
project: str                        # project name
sample_window: int                  # The sampling window for the data that flows into the TSDB and the KV
tsdb_batching_max_events: int       # The max amount of event to batch before writing the batch to tsdb
tsdb_batching_timeout_secs: int     # The max amount of seconds a given batch can be gathered before being emitted
parquet_batching_max_events: int    # The max amount of event to batch before writing the batch to parquet
parquet_batching_timeout_secs: int  # The max amount of seconds, a given batch can be gathered before being written to parquet
aggregate_count_windows: List[str]  # List of window sizes for predictions count
aggregate_count_period: str         # Period of predictions count windows
aggregate_avg_windows: List[str]    # List of window sizes for average latency
aggregate_avg_period: str           # Period of average latency windows
v3io_access_key: str                # V3IO Access key, if not set will be taken from environment
v3io_framesd: str                   # V3IO framesd URL, if not set will be taken from environment
```

In [ ]:
# Set project name
project = ""

## Deploy Model Servers

Use the following code to deploy the model servers.

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris

from mlrun import import_function, get_dataitem
from mlrun import projects
from mlrun.platforms import auto_mount

proj = projects.new_project(project)

get_dataitem("https://s3.wasabisys.com/iguazio/models/iris/model.pkl").download(
    "model.pkl")

iris = load_iris()
train_set = pd.DataFrame(iris['data'],
                         columns=['sepal_length_cm', 'sepal_width_cm',
                                  'petal_length_cm', 'petal_width_cm'])

model_names = [
    "sklearn_ensemble_RandomForestClassifier",
    "sklearn_linear_model_LogisticRegression",
    "sklearn_ensemble_AdaBoostClassifier"
]

serving_fn = import_function('hub://v2_model_server').apply(auto_mount())

for name in model_names:
    proj.log_model(name, model_file="model.pkl", training_set=train_set)
    serving_fn.add_model(name,
                         model_path=f"store://models/{project}/{name}:latest")

serving_fn.metadata.project = project
serving_fn.set_tracking()
serving_fn.deploy()

## Deploy Stream Processing

Use the following code to deploy the stream processing.

In [ ]:
import os

from mlrun import import_function
from mlrun.platforms import mount_v3io
from mlrun.runtimes import RemoteRuntime
import json

fn: RemoteRuntime = import_function("hub://model_monitoring_stream")

fn.add_v3io_stream_trigger(
    stream_path=f"projects/{project}/model-endpoints/stream",
    name="monitoring_stream_trigger",
)

fn.set_env("MODEL_MONITORING_PARAMETERS", json.dumps(
    {"project": project, "v3io_framesd": os.environ.get("V3IO_FRAMESD")}))

fn.metadata.project = project
fn.apply(mount_v3io())
fn.deploy()

## Deploy Batch Processing

Use the following code to deploy batch processing.

In [ ]:
from mlrun import import_function
from mlrun.platforms import mount_v3io
from mlrun.runtimes import KubejobRuntime

fn: KubejobRuntime = import_function("hub://model_monitoring_batch")
fn.metadata.project = project
fn.apply(mount_v3io())
fn.run(name='model-monitoring-batch', schedule="0 */1 * * *",
       params={"project": project})

## Simulating Requests

Use the following code to stimulate requests and view data in the model monitoring feature.

In [ ]:
import json
from time import sleep
from random import choice, uniform
from sklearn.datasets import load_iris

iris = load_iris()
iris_data = iris['data'].tolist()

while True:
    for name in model_names:
        data_point = choice(iris_data)
        serving_fn.invoke(f'v2/models/{name}/infer',
                          json.dumps({'inputs': [data_point]}))
        sleep(uniform(0.1, 0.4))
    sleep(uniform(0.2, 1.7))